### Dual core BM ###

This is the usal way to import the BondMachine toolkit into your python environment.

In [ ]:
import os
BONDMACHINE_DIR="/home/"+os.environ["USER"]+"/bin"
os.environ["BONDMACHINE_DIR"]=BONDMACHINE_DIR
os.environ["PATH"]=os.environ["PATH"]+":"+os.environ["BONDMACHINE_DIR"]

In [ ]:
os.environ['XILINX_HLS'] = '/tools/Xilinx/Vitis_HLS/2023.2'
os.environ['XILINX_VIVADO'] = '/tools/Xilinx/Vivado/2023.2'
os.environ['XILINX_VITIS'] = '/tools/Xilinx/Vitis/2023.2'
os.environ['PATH']=os.environ["PATH"]+":"+os.environ['XILINX_HLS']+"/bin:"+os.environ['XILINX_VIVADO']+"/bin:"+os.environ['XILINX_VITIS']+"/bin:"

Let's import also some other useful libraries as display.

In [ ]:
from IPython import display

In [ ]:
!bmhelper doctor

Now let's write 2 assembly programs, one for each core. The first one will be a simple program that increments a counter, writes it to the output port and then jumps back to the beginning. The second one will be a program that reads the input port, multiplies it by 2 and writes it to the output port and then jumps back to the beginning.

In [ ]:
%%bash
cat <<EOF > program1.asm
clr r0
inc r0
r2o r0 o0
r2o r0 o1
j 1
EOF

cat <<EOF > program2.asm
clr r0
rset r1 2
i2r r0 i0
mult r0 r1
r2o r0 o0
j 2
EOF

List of used instructions and their descriptions:

| Instruction | Description |
|-------------|-------------|
| clr         | Clear the value of a register |
| i2r         | Move a value from an input port to a register |
| inc         | Increment the value of a register |
| j           | Jump to a specified address |
| r2o         | Move a register value to an output port |
| rset        | Set the value of a register to a specified constant |

Create the two processors and load the programs into them.

In [ ]:
%%bash
procbuilder -register-size 4 -registers 2 -inputs 0 -outputs 2 -opcodes clr,inc,r2o,j  -save-machine processor1.json -input-assembly program1.asm
procbuilder -register-size 4 -registers 2 -inputs 1 -outputs 1 -opcodes clr,i2r,mult,r2o,rset,j  -save-machine processor2.json -input-assembly program2.asm

Similarly to the previous notebook, the tool created a JSON file called cp that contains all the informations to create the processor.

In [ ]:
%%bash
ls

Now, let's create a bondmachine "empty" object (a JSON file).

In [ ]:
%%bash
bondmachine -bondmachine-file bondmachine.json

... and load the processors into it.

In [ ]:
%%bash
bondmachine -bondmachine-file bondmachine.json -add-domains processor1.json
bondmachine -bondmachine-file bondmachine.json -add-domains processor2.json
bondmachine -bondmachine-file bondmachine.json -add-processor 0
bondmachine -bondmachine-file bondmachine.json -add-processor 1

The bondmachine object is a JSON file that contains all the information about the processor. It is possible to inspect it and see it.

In [ ]:
%%bash
cat bondmachine.json

In [ ]:
!bondmachine -bondmachine-file bondmachine.json -emit-dot -dot-detail 5 | dot -Tpng > bondmachine.png
display.Image("bondmachine.png")

Let's add two output ports to the bondmachine object.

In [ ]:
%%bash
bondmachine -bondmachine-file bondmachine.json -add-outputs 2

In [ ]:
!bondmachine -bondmachine-file bondmachine.json -emit-dot -dot-detail 5 | dot -Tpng > bondmachine.png
display.Image("bondmachine.png")

Let's connect the first output port of the first processor to the first output port of the bondmachine object.

In [ ]:
%%bash
bondmachine -bondmachine-file bondmachine.json -add-bond p0o0,o0

In [ ]:
!bondmachine -bondmachine-file bondmachine.json -emit-dot -dot-detail 5 | dot -Tpng > bondmachine.png
display.Image("bondmachine.png")

Now let's connect the first output port of the second processor to the second output port of the bondmachine object.

In [ ]:
%%bash
bondmachine -bondmachine-file bondmachine.json -add-bond p1o0,o1

In [ ]:
!bondmachine -bondmachine-file bondmachine.json -emit-dot -dot-detail 5 | dot -Tpng > bondmachine.png
display.Image("bondmachine.png")

And let's connect the second output port of the first processor to the input port of the second processor.

In [ ]:
%%bash
bondmachine -bondmachine-file bondmachine.json -add-bond p0o1,p1i0

In [ ]:
!bondmachine -bondmachine-file bondmachine.json -emit-dot -dot-detail 5 | dot -Tpng > bondmachine.png
display.Image("bondmachine.png")